In [19]:
#importing Libraries
import pandas as pd
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import numpy as np
import time
import os
import urllib
import re

## Q.1 Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [20]:
item=input('Enter Your Product:- ')
driver = webdriver.Chrome()
driver.get('https://www.amazon.in')
search_box = driver.find_element_by_id('twotabsearchtextbox').send_keys(item)
search_button = driver.find_element_by_id("nav-search-submit-text").click()

Enter Your Product:- guitar



## Q.2 In the above question, now scrape the following details of each product listed in first 3 pages

In [21]:
def search_amazon(item):

    #number of pages
    num = np.arange(1,4)
    
    #Making List
    ref=[]
    brands = []
    names = [] 
    ratings = []
    num_ratings =[]
    prices = []
    exchanges = []
    expected = []
    availability =[]
    others = []
    product_url = []
    
    for page in num:
        urls=driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']/div[2]/h2/a")
        nxt_button=driver.find_element_by_xpath("//li[@class='a-last']/a")
        next_arr= nxt_button.get_attribute('href')
        for i in urls:
            ref.append(i.get_attribute('href'))
        for url in ref:
            driver.get(url)
            time.sleep(2)
            #Scrapping Brand Name
            try:
                if driver.find_element_by_xpath("//th[@class='a-color-secondary a-size-base prodDetSectionEntry']").text == 'Brand':
                    brand = driver.find_element_by_xpath("//td[@class='a-size-base prodDetAttrValue']")
                    brands.append(brand.text) 
                else:
                    brand = driver.find_element_by_xpath("//td[@class='a-span9']")
                    brands.append(brand.text)
            except NoSuchElementException as e:
                brands.append("--")

            #Scrapping Product Info
            try:
                product = driver.find_element_by_id('productTitle')
                names.append(product.text)
            except NoSuchElementException as e:
                names.append("--")

            #Scrapping Ratings
            try:
                rating = driver.find_element_by_xpath("//span[@class='a-size-base a-nowrap']/span")
                ratings.append(rating.text)
            except NoSuchElementException as e:
                ratings.append("--")

            # Scrapping Number of Ratings
            try:
                num_rating = driver.find_element_by_id('acrCustomerReviewText')
                num_ratings.append(num_rating.text)
            except NoSuchElementException as e:
                num_ratings.append("--")

            #Scrapping Replacement
            try:
                exchange = driver.find_element_by_xpath("//div[@id='RETURNS_POLICY']")
                exchanges.append(exchange.text)
            except NoSuchElementException as e:
                exchanges.append("--")

            #Scrapping Price
            try:
                price = driver.find_element_by_xpath("//td[@class='a-span12']/span[1]")
                prices.append(price.text)
            except NoSuchElementException as e:
                prices.append("--")


            #Scrapping Delivery
            try:
                expect = driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b")
                expected.append(expect.text)
            except NoSuchElementException as e:
                expected.append("--")


             #Scrapping Availability
            try:
                avail = driver.find_element_by_xpath("//div[@id='availability']")
                availability.append(avail.text)
            except NoSuchElementException as e:
                availability.append("--")

            #Scrapping Other Details
            try:
                other = driver.find_element_by_xpath("//div[@id='featurebullets_feature_div']")
                others.append(other.text.replace('\n', ''))
            except NoSuchElementException as e:
                others.append("--")

            #Screpping URL
            product_url.append(url)


        driver.get(next_arr)
        time.sleep(2)
        print("Number of pages :-", page)
    #making dataframe
    ds = pd.DataFrame({'Brand' : brands ,'names' : names ,
                         'ratings' :ratings ,'num_ratings' :num_ratings ,
                         'prices' :prices ,'exchanges' :exchanges ,
                         'expected' :expected ,'availability' :availability ,
                         'others' :others ,'product_url' :product_url})
    return ds

In [22]:
product_search = search_amazon(item)


Number of pages :- 1
Number of pages :- 2
Number of pages :- 3


In [23]:
# Saving the data frame into csv
product_search.to_csv('guitar.csv')

In [24]:
product_search

,Brand,names,ratings,num_ratings,prices,exchanges,expected,availability,others,product_url
0,Guitar Bro,GUITAR BRO - Guitar Learning Combo (30 Day Lea...,3.2 out of 5,6 ratings,"₹ 2,499.00",7 Days Replacement,"Sunday, Feb 28",In stock.,30 Day Learning KitAcoustic Guitars Accessorie...,https://www.amazon.in/gp/slredirect/picassoRed...
1,VD Enterprise,VD ENTERPRISE String Acoustic Guitar | Adjusta...,5 out of 5,1 rating,₹ 749.00,7 Days Replacement,"Monday, March 1",In stock.,Toy Guitar is a Great Creativity for little Ki...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Juârez,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",3.9 out of 5,"7,303 ratings","₹ 2,397.00",7 Days Replacement,"Sunday, Feb 28",In stock.,"About this itemBlack Glossy Finish, Number of ...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
3,JUAREZ,"Juarez Acoustic Guitar, 38 Inch Curved Body Cu...",3.9 out of 5,516 ratings,"₹ 2,499.00",7 Days Replacement,"Saturday, Feb 27",In stock.,"About this itemJumbo Design, 38 Inch Acoustic ...",https://www.amazon.in/Juarez-Acoustic-Guitar-C...
4,Intern,"Intern INT-38C Acoustic Guitar Kit, With Bag, ...",3.9 out of 5,"4,213 ratings","₹ 2,290.00",7 Days Replacement,"Friday, Feb 26",In stock.,About this itemGreat looks with an innovative ...,https://www.amazon.in/Intern-INT-38C-Acoustic-...
...,...,...,...,...,...,...,...,...,...,...
303,JUAREZ,"Juarez JRZ-ST02 6-String Electric Guitar, Righ...",4.1 out of 5,100 ratings,"₹ 5,749.00",7 Days Replacement,"Friday, Feb 26",In stock.,About this itemNumber of Frets: 18Great looks ...,https://www.amazon.in/Juarez-JRZ-ST01-6-String...
304,Granada,"Granada, Acoustic Guitar, Cutaway, Medium PRS-...",--,--,"₹ 7,303.00",7 Days Replacement,"Friday, Feb 26",In stock.,"About this itemTop, Back & Sides is built of f...",https://www.amazon.in/Granada-Acoustic-PRS-1-P...
305,YAMAHA,Yamaha TRBX174 TRBX Series Electric Bass Guita...,4.7 out of 5,3 ratings,"₹ 16,625.00",7 Days Replacement,"Tuesday, March 2",Only 2 left in stock.,About this itemMaple neckSnorkeling fingerboar...,https://www.amazon.in/Yamaha-TRBX174-TRBX-Elec...
306,ARCTIC,ARCTIC Vigor Acoustic Guitar package with 40 i...,--,--,"₹ 1,379.00",7 Days Replacement,--,In stock.,About this itemMade with the quality wood with...,https://www.amazon.in/Vigor-Acoustic-Guitar-pa...


## Q.3  Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [10]:
from selenium import webdriver
import os
import urllib
import time
import pandas as pd

url_prefix = "https://www.google.com.sg/search?q="
url_postfix = "&source=lnms&tbm=isch&sa=X&ei=0eZEVbj3IJG5uATalICQAQ&ved=0CAcQ_AUoAQ&biw=939&bih=591"

save_folder = 'Train Images'

def main():
    if not os.path.exists(save_folder):
        os.mkdir(save_folder)
    download_images()
    
def download_images():
    link=[]
    name=[]
    topic = input("What do you want to search for? ")
    
    n_images = int(input('How many images do you want? '))
    
    search_url = url_prefix+topic+url_postfix
    #print(search_url)
    
    driver = webdriver.Chrome()
    driver.get(search_url)
    
    value = 0
    for i in range(5):
        driver.execute_script("scrollBy("+ str(value) +",+1000);")
        value += 1000
        time.sleep(1)
    
    elem1 = driver.find_element_by_id('islmp')
    sub = elem1.find_elements_by_tag_name('img')
    
    count=0
    for j,i in enumerate(sub):
        if j < n_images:
            src = i.get_attribute('src')                         
            try:
                if src != None:
                    src  = str(src)
                    link.append(src)
                    name.append(topic+str(j)+'.jpg')
                    urllib.request.urlretrieve(src, os.path.join(save_folder, topic+str(j)+'.jpg'))
                else:
                    raise TypeError
            except Exception as e:              #catches type error along with other errors
                print(f'fail with error {e}')
    # print("Check Folder for images")
    df=pd.DataFrame({'Name':name,'URL':link})
    print(df)
    driver.close()

In [11]:
# Images of cars
if __name__ == "__main__":
    main()

What do you want to search for? cars
How many images do you want? 100
          Name                                                URL
0    cars0.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
1    cars1.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
2    cars2.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
3    cars3.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
4    cars4.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
..         ...                                                ...
95  cars95.jpg  https://encrypted-tbn0.gstatic.com/images?q=tb...
96  cars96.jpg  https://encrypted-tbn0.gstatic.com/images?q=tb...
97  cars97.jpg  https://encrypted-tbn0.gstatic.com/images?q=tb...
98  cars98.jpg  https://encrypted-tbn0.gstatic.com/images?q=tb...
99  cars99.jpg  https://encrypted-tbn0.gstatic.com/images?q=tb...

[100 rows x 2 columns]


In [12]:
# Images of fruits
if __name__ == "__main__":
    main()

What do you want to search for? fruits
How many images do you want? 100
            Name                                                URL
0    fruits0.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
1    fruits1.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
2    fruits2.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
3    fruits3.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
4    fruits4.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
..           ...                                                ...
95  fruits95.jpg  https://encrypted-tbn0.gstatic.com/images?q=tb...
96  fruits96.jpg  https://encrypted-tbn0.gstatic.com/images?q=tb...
97  fruits97.jpg  https://encrypted-tbn0.gstatic.com/images?q=tb...
98  fruits98.jpg  https://encrypted-tbn0.gstatic.com/images?q=tb...
99  fruits99.jpg  https://encrypted-tbn0.gstatic.com/images?q=tb...

[100 rows x 2 columns]


In [13]:
# Images of Machine Learning
if __name__ == "__main__":
    main()

What do you want to search for? Machine Learning
How many images do you want? 100
                      Name                                                URL
0    Machine Learning0.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
1    Machine Learning1.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
2    Machine Learning2.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
3    Machine Learning3.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
4    Machine Learning4.jpg  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
..                     ...                                                ...
95  Machine Learning95.jpg  https://encrypted-tbn0.gstatic.com/images?q=tb...
96  Machine Learning96.jpg  https://encrypted-tbn0.gstatic.com/images?q=tb...
97  Machine Learning97.jpg  https://encrypted-tbn0.gstatic.com/images?q=tb...
98  Machine Learning98.jpg  https://encrypted-tbn0.gstatic.com/images?q=tb...
99  Machine Learning99.jpg  https://encrypted-tbn0.gstatic.c

## Q.4 Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page.

In [10]:
def main():
    
    #taking input
    item = input("Enter name of Mobile :- ")
    
    #Searching URL
    driver.get('https://www.flipkart.com')

    #clicking Pop-up Cross Button
    try:
        cross_btn = driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']")
        cross_btn.click() 
    except NoSuchElementException as e:
        pass
    
    #finding  different element
    search = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    search.send_keys( item + 'mobile')

    #clicking Search button
    search_btn = driver.find_element_by_xpath("//button[@class='L0Z3Pu']")
    search_btn.click()
    
    #Making List
    ref=[]
    brands = []
    names = [] 
    colors = []
    rams =[]
    storages = []
    pcameras = []
    scameras = []
    dsizes =[]
    dres = []
    process = []
    process_cores = []
    batteries =[]
    prices = []
    product_url = []


    time.sleep(2)
    value = 0
    for i in range(10):
        driver.execute_script("scrollBy("+ str(value) +",+1000);")
        value += 1000

    urls=driver.find_elements_by_xpath("//div[@class='_1AtVbE col-12-12']/div/div/div/a")
    
    for i in urls:
        ref.append(str(i.get_attribute('href')))
    for url in ref:
        driver.get(url)
        time.sleep(2)
        driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
        #Scrapping Brand Name
        try:
            brand = driver.find_element_by_xpath("//h1[@class='yhB1nd']/span")
            brands.append(brand.text.split()[0])
        except NoSuchElementException as e:
            brands.append("-")

        #Scrapping Model Number
        try:
            product = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[1]/table/tbody/tr[3]/td[2]")
            names.append(product.text)
        except NoSuchElementException as e:
            names.append("-")

        #Scrapping Color
        try:
            color = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[1]/table/tbody/tr[4]/td[2]")
            colors.append(color.text)
        except NoSuchElementException as e:
            colors.append("-")

        # Scrapping RAM
        try:
            ram = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[4]/table/tbody/tr[2]/td[2]")
            rams.append(ram.text)
        except NoSuchElementException as e:
            rams.append("-")

        # Scrapping ROM
        try:
            storage = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[4]/table/tbody/tr[1]/td[2]")
            storages.append(storage.text)
        except NoSuchElementException as e:
            storages.append("-")

        # Scrapping Primary Camera
        try:
            pcamera = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[5]/table/tbody/tr[2]/td[2]")
            pcameras.append(pcamera.text)
        except NoSuchElementException as e:
            pcameras.append("-")

        # Scrapping Secondary Camera
        try:
            scamera = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[5]/table/tbody/tr[5]/td[2]")
            scameras.append(scamera.text)
        except NoSuchElementException as e:
            scameras.append("-")

        #Scrapping Display Size
        try:
            dsize = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[2]/table/tbody/tr[1]/td[2]")
            dsizes.append(dsize.text)
        except NoSuchElementException as e:
            dsizes.append("-")

        #Scrapping Display Resolution
        try:
            dre = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[2]/table/tbody/tr[2]/td[2]")
            dres.append(dre.text)
        except NoSuchElementException as e:
            dres.append("-")


        #Scrapping Processor Name
        try:
            proces = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[3]/table/tbody/tr[2]/td[2]")
            process.append(proces.text)
        except NoSuchElementException as e:
            process.append("-")


        #Scrapping Processor Number of Cores
        try:
            process_core = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[3]/table/tbody/tr[3]/td[2]")
            process_cores.append(process_core.text)
        except NoSuchElementException as e:
            process_cores.append("-")

        #Scrapping Battery
        try:
            if driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[10]/div").text == 'Battery & Power Features':
                battery = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[10]/table/tbody/tr[1]/td[2]")
                batteries.append(battery.text)
            else:
                batteries.append("-")
        except NoSuchElementException as e:
            batteries.append("-")

        #Scrapping Price
        try:
            price = driver.find_element_by_xpath("//div[@class='dyC4hf']/div[1]/div/div[1]")
            prices.append(price.text)
        except NoSuchElementException as e:
            prices.append("-")

        #Screpping URL
        product_url.append(url)
        
    #making dataframe
    ds = pd.DataFrame({'Brand' : brands ,'Names' : names ,'Color':colors ,'RAM': rams, 'ROM' : storages, 
                       'Primary Camera' : pcameras, 'Secondary Camera' : scameras,'Display Size':dsizes, 
                       'Display Resulation':dres, 'Processor':process, 'Core':process_cores, 'Battery':batteries,
                       'Price':prices,'product_url' :product_url})
    
    
    
    #saving dataframe to csv
    ds.to_csv('mobile_list.csv')
    
    return ds
if __name__ == "__main__":
    main() 

df= pd.read_csv('mobile_list.csv')
df

Enter name of Mobile :- Pixel 4a


,Unnamed: 0,Brand,Names,Color,RAM,ROM,Primary Camera,Secondary Camera,Display Size,Display Resulation,Processor,Core,Battery,Price,product_url
0,0,Google,Pixel 4a,Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 730G,Octa Core,-,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,1,Google,Pixel 3a,Clearly White,4 GB,64 GB,12.2MP Rear Camera,8MP Front Camera,14.22 cm (5.6 inch),2220 x 1080 pixels,Qualcomm Snapdragon 670,Octa Core,-,"₹30,999",https://www.flipkart.com/google-pixel-3a-clear...
2,2,Trio,T4 Power,Black,32 MB,32 MB,0.08MP Rear Camera,Yes,4.5 cm (1.77 inch),128 x 160$$pixel,SC6531E/ 312MHZ,1.2 MHz,2500 mAh,₹949,https://www.flipkart.com/trio-t4-power/p/itme6...
3,3,Trio,T4 Power,Red,32 MB,32 MB,0.08MP Rear Camera,Yes,4.5 cm (1.77 inch),128*160,1.2 MHz,-,2500 mAh,₹949,https://www.flipkart.com/trio-t4-power/p/itme6...
4,4,Micromax,-,-,Quad Core,Android Nougat 7.0,2 GB,-,"Handset,Battery,Charger,User Guide",q440,1280 x 720$$pixel,-,3000 mAh,"₹5,590",https://www.flipkart.com/micromax-bharat-4-plu...
5,5,GIVA,G4,Black,0.3MP Rear Camera,Yes,Yes,-,4.57 cm (1.8 inch),128 x 160$$pixel,32 MB,Dedicated Slot,-,₹630,https://www.flipkart.com/giva-g4/p/itmd73112de...
6,6,Tecno,Spark 4 Air,Polar Blue,3 GB,32 GB,13MP + 0.3MP,5MP Front Camera,15.49 cm (6.1 inch),720 x 1560$$pixel,MediaTek Helio A22,Octa Core,-,"₹6,499",https://www.flipkart.com/tecno-spark-4-air-pol...
7,7,Redmi,Redmi 9 Prime,Sunrise Flare,4 GB,64 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh,"₹9,499",https://www.flipkart.com/redmi-9-prime-sunrise...
8,8,Redmi,Redmi 9 Prime,Matte Black,4 GB,64 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh,"₹9,499",https://www.flipkart.com/redmi-9-prime-matte-b...
9,9,Redmi,Redmi 9 Power,Blazing Blue,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹10,985",https://www.flipkart.com/redmi-9-power-blazing...


## Q.5 Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [16]:
import selenium
import pandas as pd
from selenium import webdriver

City=input("Enter City:- ")
driver=webdriver.Chrome()
driver.get('https://www.google.com/maps')
driver.find_element_by_id('searchboxinput').send_keys(City)
src_btn=driver.find_element_by_id('searchbox-searchbutton').click()

Enter City:- New Delhi


In [18]:
current_url=driver.current_url
 
Lat=float(re.findall("(?<=@)\d{1,3}\.\d{5,}",str(current_url))[0])
Long= float(re.findall("(?<=,)\d{1,3}\.\d{5,}",str(current_url))[0])
    
print('Latitide :',Lat)
print('Longitude:',Long)

Latitide : 28.5272803
Longitude: 77.068899


## Q.6 Write a program to scrap details ofall the funding dealsfor second quarter (i.e. July 20 –September 20) from trak.in.

In [10]:
import selenium
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

driver=webdriver.Chrome()
driver.get('https://trak.in/india-startup-funding-investment-2015/')

html=driver.page_source
df=pd.read_html(html)

In [17]:
print("July 2020")
df[6]

July 2020


,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,1200000000
1,2,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,100000000
2,3,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,934160
3,4,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,1700000
4,5,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,3300000
5,6,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,400000
6,7,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,974200
7,8,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,292800
8,9,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,200000
9,10,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,500000


In [19]:
print("August 2020")
df[5]

August 2020


,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,15/08/2020,Practo,HealthTech,Health care and Wellness,Bangalore,A1A Company,Series F,32000000
1,2,13/08/2020,Medlife,E-commerce,Online Pharmacy,Bangalore,Prasid Uno Family Trust and SC Credit Fund,NaN,23000000
2,3,13/08/2020,HungerBox,FoodTech,Online Food Delivery Service,Bangalore,"One97, Sabre Partners Trust, Pratithi Investme...",Series D1,1560000
3,4,04/08/2020,Dunzo,Hyper-local Logistics,Online Delivery Services,Bangalore,Existing Backers,In Progress,30000000
4,5,11/08/2020,Terra.do,EduTech,"Online Climate School, E-learning","Stanford, California,","Stanford Angels and Entrepreneurs (India), BEE...",Seed,1400000
5,6,12/08/2020,Classplus,EduTech,"E-learning, Online Tutoring",Noida,Falcon Edge,In Progress,"upto 15,000,000"
6,7,14/08/2020,Niyo,FinTech,Financial Services,Bangalore,Niyo Solutions Inc.,NaN,6000000
7,8,10/08/2020,ZestMoney,FinTech,Financial Services,Bangalore,Primrose Hills Ventures,NaN,10670000
8,9,07/08/2020,FreshToHome,E-commerce,Food Delivery,Bangalore,Ascent Capital,Venture,16200000
9,10,13/08/2020,Eduvanz,FinTech,Financial Services,Mumbai,"Sequoia India, Unitus",Series A,5000000


In [20]:
print("September 2020")
df[4]

September 2020


,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,08/09/2020,Byju’s,EduTech,Online Tutoring,Bangalore,"Silver Lake, Tiger Global, General Atlantic an...",Private Equity,500000000
1,2,12/09/2020,mCaffeine,Personal Care,Skincare & Haircare,Mumbai,"Amicus Capital Private Equity I LLP, Amicus Ca...",Series B,3000000
2,3,09/09/2020,Qshala,EduTech,Online Curiosity Platform for Kids,Bangalore,Rainmatter Capital,Angel,370000
3,4,02/09/2020,Winzo,Online Gaming,Online Gaming,New Delhi,"Kalaari Capital Partners, IndigoEdge Managemen...",Series B,15500000
4,5,09/09/2020,Hippo Video,Video Customer Experience(CX) Platform,Video Customer Experience(CX) Platform,"Newark, Delaware, United States of Amercia","Alpha Wave Incubation, Exfinity Venture Partne...",Series A,4500000
5,6,07/09/2020,Melorra,E-commerce,Online Jewelry Store,Bangalore,"Shadow Holdings, Lightbox.",Debt Financing,"upto 8,900,000"
6,7,07/09/2020,1mg,E-commerce,Online Pharmacy,Gurgaon,"Gaja Capital, Tata Capital, Partners Group",In Progress,100000000
7,8,31/08/2020,mfine,HealthTech,On-Demand Healthcare Services,Bangalore,Caretech Pte Inc,Series B,5400000
8,9,31/08/2020,Apna,Human Resources,Recruitment Platform,Bangalore,Lightspeed India and Sequoia Capital India,Series A,8000000
9,10,03/09/2020,Railofy,Transportation,WL & RAC protection platform,Mumbai,Chiratae Ventures,Seed,950000


## Q.7 Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [20]:
driver=webdriver.Chrome()
#Getting Link
driver.get('https://www.digit.in/')

#Navigating to the link of best gaming laptop
l1 = driver.find_element_by_xpath("//div[@class='listing_container']/ul/li[9]/a").get_attribute('href')
driver.get(l1)

#downloading page source
html = driver.page_source

#reading tables
df = pd.read_html(html)

#Scrapping Specification Tables
num = np.arange(1,11)
l1= []
for i in num:
    df1 = df[1].drop('SPECIFICATION.1', axis = 1)
    df2 = df1.drop('SPECIFICATION', axis = 1)
    ds = df2.transpose()
    
    l1.append(ds)
    
#Joining Specification Tables
result = pd.concat(l1,ignore_index=True,join="inner")
result['Product Name']=df[0]['Product Name']
result['Seller']=df[0]['Seller']
result['Price']=df[0]['Price']

result.drop(7, axis = 1, inplace = True)

#Renaming final Table
result = result.rename(columns={0 : "OS", 1: "Display",2: "Processor",3: "Memory",4: "Weight",5: "Dimension",6: "Graphics Processor"})
result

,OS,Display,Processor,Memory,Weight,Dimension,Graphics Processor,Product Name,Seller,Price
0,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,MSI GT76 TITAN DT 9SG,NaN,₹379990
1,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,Alienware 17 Area-51m,amazon,₹489909
2,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,HP Omen 15 2020,amazon,₹114990
3,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,Asus Zephyrus G14,amazon,₹164990
4,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,Lenovo Legion Y540,flipkart,₹79990
5,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,Asus ROG Zephyrus G GA502,flipkart,₹79990
6,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,Asus ROG Zephyrus S GX531,amazon,₹146000
7,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,MSI GT83VR 7RE Titan SLI,NaN,₹349990
8,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,ASUS ROG ZEPHYRUS DUO 15,amazon,₹259990
9,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,Dell G3,amazon,₹61990
